In [3]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
num =0


In [4]:
def get_text(url):
    """
    Function to retrieve HTML text content from a given URL.

    Parameters:
    - url (str): The URL to retrieve HTML text from.

    Returns:
    - str: HTML text content from the given URL.
    """
    try:
        headers = {
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
        }
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except Exception as e:
        return f'Failed to read due to {e}'

In [5]:
# overview of source code
# get_text('https://store.steampowered.com/search/?filter=globaltopsellers&os=win')

In [6]:
def run(game_info,jump_link,text):
    """
    Function to extract game information and links from HTML text content.

    Parameters:
    - game_info (list): List to store game information.
    - jump_link (list): List to store game page links.
    - text (str): HTML text content.
    """
    
    global num
    soup = BeautifulSoup(text, "html.parser")
    
    # game page
    # the game page link are stored in "search_resultsRows", under a.href
    # code partly cited from: https://blog.csdn.net/DDDHL_/article/details/111768725
    link_text = soup.find_all("div", id="search_resultsRows")
    for text in link_text:
        search_id = text.find_all('a')
    for tags in search_id:
        jump_link.append(tags['href'])
        
        game_info.append([num + 1, jump_link[num]])
        num = num + 1

In [7]:
if __name__ == "__main__":
    Game_info = []          
    Turn_link = []          
    Jump_link = []          
    for i in range(1, 41):
        Turn_link.append("https://store.steampowered.com/search/?filter=globaltopsellers&page=1&os=win" + str("&page=" + str(i)))
        run(Game_info, Jump_link,get_text(Turn_link[i-1]))

In [8]:
# show the links to detailed game page for the first five games
Jump_link[:5]

['https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_globaltopsellers_150_1',
 'https://store.steampowered.com/app/553850/HELLDIVERS_2/?snr=1_7_7_globaltopsellers_150_1',
 'https://store.steampowered.com/app/294100/RimWorld/?snr=1_7_7_globaltopsellers_150_1',
 'https://store.steampowered.com/app/2380740/RimWorld__Anomaly/?snr=1_7_7_globaltopsellers_150_1',
 'https://store.steampowered.com/app/578080/PUBG_BATTLEGROUNDS/?snr=1_7_7_globaltopsellers_150_1']

In [9]:
def save_data(game_info):
    """
    Function to save game information to a CSV file.

    Parameters:
    - game_info (list): List containing game information.

    Returns:
    - df (DataFrame): DataFrame containing game information.
    """
    save_path = "Steam.csv"
    df = pd.DataFrame(game_info, columns=['Rank', 'Game Name', 'Current Price', 'Recent Comment', 'Release Date', 'Tags'])
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df['Tags'] = df['Tags'].str.replace(r'\t+', '', regex=True).str.replace(r'\r\n+', ',', regex=True)
    # Save DataFrame to CSV file
    df.to_csv(save_path, index=False)
    print("save success")
    return df

In [10]:
rank = 1
save_path = "Steam.csv"
game_info = []

# iterate through each link 
for i in Jump_link:
    response = requests.get(i)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # game name
        title = soup.find('title').text
        
        # original price
        discount_price = soup.find('div', class_='discount_original_price')
        if discount_price:
            price = discount_price.text
        else:
            original_price = soup.find('div', class_='game_purchase_price price')
            if original_price:
                price = original_price.text
            else:
                price = 0
        
        # recent reviews
        review_30days = soup.find('div', class_='summary column')
        if review_30days:
            review_recent = review_30days.find('span', class_='nonresponsive_hidden responsive_reviewdesc')
            if review_recent:
                review_recent = review_recent.text
            else:
                review_recent = 'N/A'
        else:
            review_recent = 'N/A'
            
            
        # release date
        date = soup.find('div', class_='release_date')
        if date:
            date = date.text
        else:
            date = 'NA'
        
        # tags
        tags = soup.find('div', class_='glance_tags popular_tags')
        if tags:
            tags = tags.text
        else:
            tags = 'N/A'
        
        # append game info to list
        game_info.append([rank, title, price, review_recent, date,tags])
        
        rank += 1

In [11]:
df = save_data(game_info)
df

save success


,Rank,Game Name,Current Price,Recent Comment,Release Date,Tags
0,1,Counter-Strike 2 on Steam,Free to Play,"- 77% of the 65,534 user reviews in the last 3...","Release Date:\nAug 21, 2012","FPS,Shooter,Multiplayer,Competitive,Action,Tea..."
1,2,HELLDIVERS™ 2 on Steam,$39.99,"- 90% of the 79,601 user reviews in the last 3...","Release Date:\nFeb 8, 2024","Online Co-Op,Action,Third-Person Shooter,Multi..."
2,3,Save 20% on RimWorld on Steam,$34.99,"- 97% of the 2,691 user reviews in the last 30...","Release Date:\nOct 17, 2018","Colony Sim,Base Building,Survival,Strategy,San..."
3,4,RimWorld - Anomaly on Steam,$59.98,- 96% of the 765 user reviews for this game ar...,"Release Date:\nApr 11, 2024","Colony Sim,Base Building,Survival,Strategy,San..."
4,5,PUBG: BATTLEGROUNDS on Steam,Free to Play,"- 69% of the 21,476 user reviews in the last 3...","Release Date:\nDec 21, 2017","Survival,Shooter,Battle Royale,Multiplayer,FPS..."
...,...,...,...,...,...,...
995,996,Mind Over Magic on Steam,$24.99,- 89% of the 89 user reviews in the last 30 da...,"Release Date:\nDec 14, 2023","Colony Sim,Base Building,Immersive Sim,Managem..."
996,997,Travellers Rest on Steam,$17.99,- 85% of the 351 user reviews in the last 30 d...,"Release Date:\nJul 28, 2020","Life Sim,Pixel Graphics,Farming Sim,Management..."
997,998,Operation: Harsh Doorstop - Development Build ...,$19.99,- Need more user reviews to generate a score,"Release Date:\nMar 29, 2023","Action,Shooter,FPS,3D,First-Person,Military,Pv..."
998,999,Save 15% on Dealer Simulator on Steam,$13.99,- 62% of the 134 user reviews in the last 30 d...,"Release Date:\nJan 29, 2024","FPS,Immersive Sim,Casual,Simulation,Driving,Ma..."
